In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from db import get_pd_from_table
from konlpy.tag import Mecab
from tqdm import tqdm

In [2]:
books = get_pd_from_table('books')
books.drop(['isbn', 'pubDate', 'img', 'rate', 'bestseller', 'similarity'], axis=1, inplace=True)
books.dropna(axis=0, inplace=True)
books.head()

,id,title,author,publisher,summary,genre
0,1,리스트 고독한가운데 신의 축복 (984),마스트미디어 편집부 (지은이),마스트미디어,,예술/대중문화
1,2,거짓말이다 + 꽃다발 상품권 세트,김탁환 (지은이),북스피어,김탁환 작가의 &lt;거짓말이다&gt;와 꽃다발을 함께 구입할 수 있는 기획 상품....,소설/시/희곡
2,3,[헌금봉투] 주일헌금 201 (1속50매) - VJ-1311-201,비전북 편집부 (지은이),비전북(VisionBook),비전아트 고급캘리헌금봉투는 캘리그래피를 담은 우아한 디자인을 더했다. 두꺼운 재질의...,종교/역학
3,4,[헌금봉투] 주일헌금 202 (1속50매) - VJ-1311-202,비전북 편집부 (지은이),비전북(VisionBook),비전아트 고급캘리헌금봉투는 캘리그래피를 담은 우아한 디자인을 더했다. 두꺼운 재질의...,종교/역학
4,5,엘르 Elle E형 2022.8 (표지 : NCT 재현) (부록없음) - 주요기사 ...,허스트중앙 편집부 (지은이),허스트중앙(Hearst-Joongang),,잡지


In [3]:
summaries = []
for book in books.values:
    summary = book[1]
    summary += book[2]
    summary += book[3]
    summary += book[4]
    summary += book[5]
    summaries.append(summary)
books['summary'] = summaries
books.drop(['title','author','publisher','genre'], axis=1, inplace=True)
books.head()

,id,summary
0,1,리스트 고독한가운데 신의 축복 (984)마스트미디어 편집부 (지은이)마스트미디어예술...
1,2,거짓말이다 + 꽃다발 상품권 세트김탁환 (지은이)북스피어김탁환 작가의 &lt;거짓말...
2,3,[헌금봉투] 주일헌금 201 (1속50매) - VJ-1311-201비전북 편집부 (...
3,4,[헌금봉투] 주일헌금 202 (1속50매) - VJ-1311-202비전북 편집부 (...
4,5,엘르 Elle E형 2022.8 (표지 : NCT 재현) (부록없음) - 주요기사 ...


In [4]:
# 불용어 가져오기
stopwords = []
with open("data/hangul_stopword.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
        line = line.replace('\n', '')
        stopwords.append(line)
print(stopwords)

['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또', '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '좀', '원', '잘', '통하', '소리', '놓']


In [5]:
# summary 전처리
books['summary'] = books['summary'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
mecab = Mecab()
corpus_list = []
for index, row in tqdm(books.iterrows(), total=len(books)):
    text = row['summary']
    tokenized_sentence = mecab.morphs(text)
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    corpus_list.append(stopwords_removed_sentence)
books['summary'] = corpus_list
books.head()

/var/folders/9d/4103_0g54_s9yr74zb4kcm340000gn/T/ipykernel_89932/3166800422.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  books['summary'] = books['summary'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
100%|██████████| 76080/76080 [00:18<00:00, 4088.00it/s]


,id,summary
0,1,"[리스트, 고독, 가운데, 신, 의, 축복, 마스트, 미디어, 편집부, 지은, 마스..."
1,2,"[거짓말, 다, 꽃다발, 상품권, 세트, 김탁환, 지은, 북, 스피어, 김탁환, 작..."
2,3,"[헌, 금봉, 투, 주일, 헌금, 매, 비, 전북, 편집부, 지은, 비, 전북, 비..."
3,4,"[헌, 금봉, 투, 주일, 헌금, 매, 비, 전북, 편집부, 지은, 비, 전북, 비..."
4,5,"[엘르, 형, 표지, 재현, 부록, 음, 주요, 기사, 재현, 김, 태리, 아이, ..."


In [ ]:
word_model = Word2Vec.load('data/ko.bin')
a = word_model.wv.most_similar("계발")

# genre 벡터로 만들어서 추가
genre_vec = []
for genre in item_meta['genre']:
    if genre == '경제경영':
        genre_vec.append(tuple(word_model["경제"]))
    elif genre == '소설/시/희곡':
        genre_vec.append(tuple(word_model["소설"]))
    elif genre == '자기계발':
        genre_vec.append(tuple(word_model["계발"]))
    else:
        genre_vec.append(tuple(word_model[genre]))
item_meta['genre_vec'] = genre_vec
item_meta